# **Klasifikasi Sampah dengan Convolutional Neural Network (CNN)**
---

In [1]:
# File operations and data manipulation
import os, zipfile, shutil
from google.colab import drive
from shutil import copyfile

# Random number generation and numerical computations
import random
import numpy as np

# Visualization library
import matplotlib.pyplot as plt

# Deep learning framework and its modules
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# Ignore warnings during execution
import warnings
warnings.filterwarnings("ignore")

# Print TensorFlow version
print(tf.__version__)

2.14.0


## **Dataset Collection**
---

In [2]:
!wget --no-check-certificate \
  https://github.com/mrgsrylm/KlasifikasiSampah/releases/download/v1.0.0/kategori_sampah.zip \
  -O kategori_sampah.zip

!unzip kategori_sampah.zip

Streaming output truncated to the last 5000 lines.
  inflating: kategori_sampah/anorganik/anorganik6407.jpg  
  inflating: kategori_sampah/anorganik/anorganik5687.jpg  
  inflating: kategori_sampah/anorganik/anorganik3924.jpg  
  inflating: kategori_sampah/anorganik/anorganik2258.jpg  
  inflating: kategori_sampah/anorganik/anorganik1777.jpg  
  inflating: kategori_sampah/anorganik/anorganik10618.jpg  
 extracting: kategori_sampah/anorganik/anorganik2716.jpg  
  inflating: kategori_sampah/anorganik/anorganik876.jpg  
  inflating: kategori_sampah/anorganik/anorganik1861.jpg  
  inflating: kategori_sampah/anorganik/anorganik2870.jpg  
  inflating: kategori_sampah/anorganik/anorganik537.jpg  
  inflating: kategori_sampah/anorganik/anorganik1309.jpg  
  inflating: kategori_sampah/anorganik/anorganik627.jpg  
  inflating: kategori_sampah/anorganik/anorganik5624.jpg  
  inflating: kategori_sampah/anorganik/anorganik7212.jpg  
  inflating: kategori_sampah/anorganik/anorganik6488.jpg  
  infla

## **Data Pre-Processing**
---

### Settings Hyperparameter
---

In [3]:
# Constants for data splitting (T:90,V:10), batch size, input size (image dimensions), number of epochs, and data sample size
SPLIT_SIZE = 0.9
BATCH_SIZE = 32
INPUT_SIZE = (150, 150)
EPOCH = 20
N = 16

### Training & Validation Directories
---

In [4]:
DATASET_DIR = "dataset"
categories_dir = ["TRAINING", "VALIDATION"]
subcategories_dir = ["organik", "anorganik"]

for category in categories_dir:
    category_dir = os.path.join(DATASET_DIR, category)
    os.makedirs(category_dir)
    for subcategory in subcategories_dir:
        os.makedirs(os.path.join(category_dir, subcategory))

def list_directories(root_path):
  for rootdir, dirs, files in os.walk(root_path):
    for subdir in dirs:
      print(os.path.join(rootdir, subdir))

list_directories(DATASET_DIR)

dataset/VALIDATION
dataset/TRAINING
dataset/VALIDATION/anorganik
dataset/VALIDATION/organik
dataset/TRAINING/anorganik
dataset/TRAINING/organik


### Splitting Dataset

In [5]:
# split_data function: This function splits data from the source directory into training and validation sets
def split_data(SOURCE, TRAIN, VALIDATION, SPLIT_SIZE):
    # Get a list of non-empty files in the source director
    files = [filename for filename in os.listdir(SOURCE) if os.path.getsize(os.path.join(SOURCE, filename)) > 0]

    # Shuffle the list of files
    shuffle_set = random.sample(files, len(files))

    # Calculate the number of files for training and validation sets based on the split size
    training_len = int(len(files) * SPLIT_SIZE)
    validation_len = len(files) - training_len

    # Divide the shuffled files into training and validation sets
    training_set = shuffle_set[:training_len]
    validation_set = shuffle_set[-validation_len:]

    for filename in training_set:
        source = os.path.join(SOURCE, filename)
        destiny = os.path.join(TRAIN, filename)
        copyfile(source, destiny)

    for filename in validation_set:
        source = os.path.join(SOURCE, filename)
        destiny = os.path.join(VALIDATION, filename)
        copyfile(source, destiny)

ORGANIK_DATASOURCE = "kategori_sampah/organik"
ANORGANIK_DATASOURCE = "kategori_sampah/anorganik"

TRAIN_DIR = "dataset/TRAINING"
VALIDATION_DIR = "dataset/VALIDATION"

ORGANIC_TRAIN_DIR = os.path.join(TRAIN_DIR, "organik/")
ORGANIC_VALIDATION_DIR = os.path.join(VALIDATION_DIR, "organik/")

RECYCLABLE_TRAIN_DIR = os.path.join(TRAIN_DIR, "anorganik/")
RECYCLABLE_VALIDATION_DIR = os.path.join(VALIDATION_DIR, "anorganik/")

# Clear existing file
for directory in [ORGANIC_TRAIN_DIR, ORGANIC_VALIDATION_DIR, RECYCLABLE_TRAIN_DIR, RECYCLABLE_VALIDATION_DIR]:
    for file in os.scandir(directory):
        os.remove(file.path)

# Do splitting and move data
split_data(ORGANIK_DATASOURCE, ORGANIC_TRAIN_DIR, ORGANIC_VALIDATION_DIR, SPLIT_SIZE)
split_data(ANORGANIK_DATASOURCE, RECYCLABLE_TRAIN_DIR, RECYCLABLE_VALIDATION_DIR, SPLIT_SIZE)

# Print the number of files in each directory
print(f"\nThere are {len(os.listdir(ORGANIC_TRAIN_DIR))} organic material images in training")
print(f"There are {len(os.listdir(ORGANIC_VALIDATION_DIR))} organic material images in validation")
print(f"There are {len(os.listdir(RECYCLABLE_TRAIN_DIR))} recyclable material images in training")
print(f"There are {len(os.listdir(RECYCLABLE_VALIDATION_DIR))} recyclable material images in validation")


There are 12569 organic material images in training
There are 1397 organic material images in validation
There are 9999 recyclable material images in training
There are 1112 recyclable material images in validation


### Training Validation Generator and Data Augmentation
---

In [6]:
# train_val_generators function: This function creates and configures data generators for training and validation data.
def train_val_generators(TRAINING_DIR, VALIDATION_DIR, BATCH_SIZE, INPUT_SIZE):

    # Training data augmentation configuration
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Validation data augmentation configuration
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # Generating batches of augmented data for training
    train_generator = train_datagen.flow_from_directory(
        directory=TRAINING_DIR,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        target_size=INPUT_SIZE
    )

    # Generating batches of non-augmented data for validation
    validation_generator = validation_datagen.flow_from_directory(
        directory=VALIDATION_DIR,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        target_size=INPUT_SIZE
    )

    # Return the configured data generators for training and validation
    return train_generator, validation_generator

# TRAIN AND VALIDATION GENERATOR
TRAIN_GENERATOR, VALIDATION_GENERATOR = train_val_generators(TRAIN_DIR, VALIDATION_DIR, BATCH_SIZE, INPUT_SIZE)


Found 22568 images belonging to 2 classes.
Found 2509 images belonging to 2 classes.


## **Model Simple CNN**
---

### Model Definition
---

#### Create Model using CNN and Implement Flatten, Dropout, and Regularization
---

In [7]:
# create_model function: This function defines and compiles a convolutional neural network model.
def create_model_simple_cnn(INPUT_SHAPE, NUM_CLASSES):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')
    ])

    # Compile the model with appropriate optimizer, loss function, and metrics
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Create Model
INPUT_SHAPE = (150, 150, 3)
NUM_CLASSES = 2
model_simple_cnn = create_model_simple_cnn(INPUT_SHAPE, NUM_CLASSES)
model_simple_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

### **Training**
---

In [ ]:
# Training the model using the training generator and validating it using the validation generator
# 'EPOCH' specifies the number of epochs for training
# The 'verbose' parameter is set to 1 for detailed progress output during training
history_simple_cnn = model_simple_cnn.fit(TRAIN_GENERATOR,
                    epochs=EPOCH,
                    validation_data=VALIDATION_GENERATOR,
                    verbose=1)

Epoch 1/20
706/706 [==============================] - 214s 277ms/step - loss: 0.4337 - accuracy: 0.8131 - val_loss: 0.4059 - val_accuracy: 0.8318
Epoch 2/20
706/706 [==============================] - 194s 275ms/step - loss: 0.3874 - accuracy: 0.8350 - val_loss: 0.4001 - val_accuracy: 0.8366
Epoch 3/20
706/706 [==============================] - 197s 279ms/step - loss: 0.3641 - accuracy: 0.8469 - val_loss: 0.3677 - val_accuracy: 0.8450
Epoch 4/20
706/706 [==============================] - 156s 221ms/step - loss: 0.3568 - accuracy: 0.8505 - val_loss: 0.3676 - val_accuracy: 0.8442
Epoch 5/20
706/706 [==============================] - 153s 216ms/step - loss: 0.3405 - accuracy: 0.8589 - val_loss: 0.3870 - val_accuracy: 0.8545
Epoch 6/20
706/706 [==============================] - 153s 217ms/step - loss: 0.3297 - accuracy: 0.8631 - val_loss: 0.3259 - val_accuracy: 0.8764
Epoch 7/20
706/706 [==============================] - 153s 216ms/step - loss: 0.3236 - accuracy: 0.8679 - val_loss: 0.3056 -

### **Evaluation**
---

#### Plotting Accuracy and Model Evaluation
---

In [ ]:
acc = history_simple_cnn.history['accuracy']
val_acc = history_simple_cnn.history['val_accuracy']
loss = history_simple_cnn.history['loss']
val_loss = history_simple_cnn.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# Evaluating the model on the validation generator using the evaluate_generator method
# 'steps' parameter specifies the total number of steps (batches of samples) to yield from validation_generator before stopping
# 'verbose=0' means silent mode, no progress bar will be displayed during evaluation
_, accuracy = model_simple_cnn.evaluate_generator(VALIDATION_GENERATOR, steps=10, verbose=0)

# Printing the test accuracy after multiplying by 100 to convert it to a percentage
print('Test Accuracy: %.3f%%' % (accuracy * 100))


### **Test**
---

In [ ]:
# Extracting class labels and creating a dictionary for label decoding
labels = VALIDATION_GENERATOR.class_indices
labels = dict((v, k) for k, v in labels.items())

# Load image
a_test_path = 'kategori_sampah/test/sampah_botol_aqua.png'
o_test_path = 'kategori_sampah/test/sampah_daun_kering.png'

a_test = image.load_img(a_test_path)
o_test = image.load_img(o_test_path)

a_result = model_simple_cnn.predict(a_test)
o_result = model_simple_cnn.predict(o_test)

plt.title('sampah :%s ' % (labels[np.argmax(a_result[0])]))
plt.imshow(a_result)

plt.title('sampah :%s ' % (labels[np.argmax(o_result[0])]))
plt.imshow(o_result)
plt.show

## **Model Inception CNN**
---

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras import layers
from keras import Model

### Model Definition
---

#### Create Model Inception using Transfer Learning
---

In [ ]:
inception_pretrained_model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False,  # Excluding the top (classification) layers
                                weights='imagenet')  # Using pre-trained weights from ImageNet

for layer in inception_pretrained_model.layers:
    layer.trainable = False

inception_pretrained_model.summary()

In [ ]:
# Getting the output from a specific layer ('mixed6') of the pre-trained InceptionV3 model
last_layer = inception_pretrained_model.get_layer('mixed6')
print('Last layer output shape:', last_layer.output_shape)  # Printing the output shape of the selected layer
last_output = last_layer.output  # Storing the output tensor of the selected layer for further processing

In [ ]:
# Adding a Flatten layer to convert the 3D tensor to 1D vector
x = layers.Flatten()(last_output)

# Adding a Dense layer with 512 units and ReLU activation function
x = layers.Dense(512, activation='relu')(x)

# Applying Dropout with a dropout rate of 0.2 to prevent overfitting
x = layers.Dropout(0.2)(x)

# Adding the final Dense layer with 2 units (for binary classification) and sigmoid activation function
predictions = layers.Dense(2, activation='sigmoid')(x)

# Creating the custom model by specifying its input (pretrained InceptionV3 model's input) and output (predictions)
model_inception = Model(inception_pretrained_model.input, predictions)

In [ ]:
# Compiling the InceptionV3-based model
model_inception.compile(optimizer=tf.keras.optimizers.Adam(0.0001),  # Using Adam optimizer with a learning rate of 0.0001
                        loss='binary_crossentropy',  # Binary crossentropy loss for binary classification task
                        metrics=['accuracy'])  # Monitoring accuracy as the evaluation metric

# Printing the summary of the compiled model architecture
model_inception.summary()

### **Training**
---

In [ ]:
# Training the model using the training generator and validating it using the validation generator
history_inception = model_inception.fit(TRAIN_GENERATOR,  # Training data generator
                                        epochs=EPOCH,  # Number of epochs for training
                                        validation_data=VALIDATION_GENERATOR,  # Validation data generator
                                        verbose=1)  # Setting verbosity level to 1 for detailed progress output during training


### **Evaluation**
---

#### Plotting Accuracy and Model Evaluation
---

In [ ]:
acc = history_inception.history['accuracy']
val_acc = history_inception.history['val_accuracy']
loss = history_inception.history['loss']
val_loss = history_inception.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

In [ ]:
_, acc = model_inception.evaluate_generator(VALIDATION_GENERATOR, steps=10, verbose=0)
print('Test Accuracy: %.3f ' % (acc * 100))

### **Prediction**
---

In [ ]:
test_x, test_y = VALIDATION_GENERATOR.__getitem__(1)

labels = (VALIDATION_GENERATOR.class_indices)
labels = dict((v,k) for k,v in labels.items())

y_pred = model_inception.predict(test_x)

plt.figure(figsize=(16, 16))
n = 16
count = 0

for i in range(n):
    plt.subplot(4, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(y_pred[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])
    if (np.argmax(y_pred[i]) == np.argmax(test_y[i])):
      count += 1

akurasi = count/n*100
print('Accuracy : {} %'.format(akurasi))

### **Store Model**
---

#### Save Model (h5)
---

In [ ]:
!mkdir -p "saved_model_inception"
!mkdir -p "tflite_model_inception"
model_inception.save('saved_model_inception/model_class_inceptionv3.h5')

#### Convert to TFLite
---

In [ ]:
tf.saved_model.save(model_inception, export_dir="saved_model_inception")

converter = tf.lite.TFLiteConverter.from_saved_model("saved_model_inception")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_inception = converter.convert()

In [ ]:
import pathlib
tflite_model_file = pathlib.Path("tflite_model_inception/model_class_inveptionv3.tflite")
tflite_model_file.write_bytes(tflite_model_inception)

## **Model Xception CNN**
---

In [ ]:
from keras.applications.xception import Xception
from keras import layers
from keras import Model

### **Model Definition**
---

#### Create Model using Transfer Learning
---

In [ ]:
xception_pretrained_model = Xception(input_shape = (150, 150, 3),
                                     include_top = False,
                                     weights = "imagenet")

In [ ]:
for layer in xception_pretrained_model.layers:
  layer.trainable = False

xception_pretrained_model.summary()

In [ ]:
last_layer = xception_pretrained_model.get_layer('add_11')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2, activation='sigmoid')(x)
model_xception = Model(xception_pretrained_model.input, x)

In [ ]:
model_xception.compile(optimizer=tf.optimizers.Adam(0.0001),
                       loss="binary_crossentropy",
                       metrics=["accuracy"])
model_xception.summary()

### **Training**
---

In [ ]:
history_xception = model_xception.fit(TRAIN_GENERATOR,
                                      epochs = EPOCH,
                                      validation_data = VALIDATION_GENERATOR,
                                      verbose = 1)

### **Evaluation**
---

#### Plotting Accuracy and Model Evaluation
---

In [ ]:
acc = history_xception.history['accuracy']
val_acc = history_xception.history['val_accuracy']
loss = history_xception.history['loss']
val_loss = history_xception.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

In [ ]:
_, acc = model_xception.evaluate_generator(VALIDATION_GENERATOR, steps=10, verbose=0)
print('Test Accuracy: %.3f ' % (acc * 100))

### **Prediction**
---

In [ ]:
test_x, test_y = VALIDATION_GENERATOR.__getitem__(1)

labels = (VALIDATION_GENERATOR.class_indices)
labels = dict((v,k) for k,v in labels.items())

y_pred = model_xception.predict(test_x)

plt.figure(figsize=(16, 16))
n = 16
count = 0
for i in range(n):
    plt.subplot(4, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(y_pred[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])
    if (np.argmax(y_pred[i]) == np.argmax(test_y[i])):
      count += 1

akurasi = count/n*100
print('Accuracy : {} %'.format(akurasi))

### **Store Model**
---

In [ ]:
!mkdir saved_model_xception
!mkdir tflite_model_xception

#### Save Model (h5)
---

In [ ]:
saved_model_path = "saved_model_xception/model_class_best.h5"
model_xception.save(saved_model_path)

#### Convert to TFLite
---

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_xception)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_xception = converter.convert()

In [ ]:
import pathlib


tflite_model_file = pathlib.Path('tflite_model_xception/model_class_best.tflite')
tflite_model_file.write_bytes(tflite_model_xception)
print("Model successfully created in .tflite!")

## Save Model to Google Drive
---

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Model Inception
source_saved_model_inception = "/content/saved_model_inception"
destination_saved_model_inception = "/content/drive/My Drive/klasifikasi_sampah/class/saved_model_inception"
shutil.copytree(source_saved_model_inception, destination_saved_model_inception)

source_tflite_model_inception = "/content/tflite_model_inception"
destination_tflite_model_inception = "/content/drive/My Drive/klasifikasi_sampah/class/tflite_model_inception"
shutil.copytree(source_tflite_model_inception, destination_tflite_model_inception)

In [ ]:
# Xception
source_saved_model_xception = "/content/saved_model_xception"
destination_saved_model_xception = "/content/drive/My Drive/klasifikasi_sampah/class/saved_model_xception"
shutil.copytree(source_saved_model_xception, destination_saved_model_xception)

source_tflite_model_xception = "/content/tflite_model_xception"
destination_tflite_model_xception = "/content/drive/My Drive/klasifikasi_sampah/class/tflite_model_xception"
shutil.copytree(source_tflite_model_xception, destination_tflite_model_xception)